# Tutorial 1.4: Manual Tracing and Advanced Observability

## Custom Instrumentation for Complex Workflows

Welcome to advanced tracing! While autologging is powerful, real-world applications often need custom instrumentation. You'll learn how to trace your own functions and build hierarchical observability into complex GenAI workflows.

### What You'll Learn
- When to use manual tracing vs. autologging
- The `@mlflow.trace` decorator
- Creating custom spans with proper types
- Adding custom attributes to spans
- Building hierarchical traces (parent-child relationships)
- Tracing RAG pipelines end-to-end
- Tracing agentic workflows with tool usage
- Advanced debugging techniques

### Prerequisites
- Completed Notebook 1.3 (Introduction to Tracing)
- Understanding of autologging
- MLflow UI running

### Estimated Time: 30-35 minutes

---
## Step 1: When to Use Manual Tracing

### Autologging is Great For:
- ✅ LLM API calls (OpenAI, Anthropic, etc.)
- ✅ Framework chains (LangChain, LlamaIndex)
- ✅ Quick prototyping
- ✅ Standard workflows

### Manual Tracing is Needed For:
- ✅ **Custom functions** in your pipeline
- ✅ **Domain-specific operations** (parsing, validation, business logic)
- ✅ **Custom retrievers** or data sources
- ✅ **External API calls** not auto-instrumented
- ✅ **Adding context** not captured automatically
- ✅ **Organizing operations** into logical groups

### Best Practice: Combine Both!
```python
# Use autologging for LLM calls
mlflow.openai.autolog()

# Add manual tracing for custom logic
@mlflow.trace
def my_custom_retriever(query):
    # Your custom code
    pass
```

---
## Step 2: Environment Setup

In [2]:
import time

import mlflow
from dotenv import load_dotenv
from utils.clnt_utils import is_databricks_ai_gateway_client, get_databricks_ai_gateway_client, get_openai_client, get_ai_gateway_model_names

# Load environment
load_dotenv()

# Configure MLflow
mlflow.set_tracking_uri("http://localhost:5000")

use_databricks_provider = is_databricks_ai_gateway_client()
if use_databricks_provider:
    client = get_databricks_ai_gateway_client()
    model_name = get_ai_gateway_model_names()[0]
else:
    # Initialize OpenAI
    client = get_openai_client()
    model_name = "gpt-5-2"
print("✅ Environment configured: using", "Databricks" if use_databricks_provider else "OpenAI", "client")
print(f"   MLflow version: {mlflow.__version__}")
print(f"   Tracking URI: {mlflow.get_tracking_uri()}")

# Create experiment
mlflow.set_experiment("07-manual-tracing")

print("✅ Environment configured")
print("   OpenAI autologging: ENABLED")
print("   Ready for manual tracing!")

✅ Environment configured: using Databricks client
   MLflow version: 3.9.0
   Tracking URI: http://localhost:5000
✅ Environment configured
   OpenAI autologging: ENABLED
   Ready for manual tracing!


---
## Step 3: Your First Manual Trace

Let's trace a custom function using the `@mlflow.trace` decorator, giving us the control what specific calls to trace as we might
have custom logic that we want trace and debug in case of issues. MLflow privdes a decorator `@mlflow.trace`

In [3]:
# Simple example: Trace a custom function
@mlflow.trace
def preprocess_query(user_query: str) -> str:
    """
    Preprocess user query before sending to LLM.
    This is a custom function that MLflow doesn't auto-trace.
    """
    # Simulate preprocessing
    time.sleep(0.1)  # Simulate work
    
    processed = user_query.strip().lower()
    
    # This function is now fully traced!
    return processed

# Test it
print("\n🔍 Testing manual tracing...\n")

result = preprocess_query("  What is MLflow?  ")

print("Input: '  What is MLflow?  '")
print(f"Output: {result}")
print("\n✅ Function traced!")
print("\n📊 In MLflow UI, you'll see:")
print("   - Span name: preprocess_query")
print("   - Input: original query")
print("   - Output: processed query")
print("   - Duration: ~100ms")


🔍 Testing manual tracing...

Input: '  What is MLflow?  '
Output: what is mlflow?

✅ Function traced!

📊 In MLflow UI, you'll see:
   - Span name: preprocess_query
   - Input: original query
   - Output: processed query
   - Duration: ~100ms


Trace(trace_id=tr-68a72e5fdacfc2da2992cc861ce85e94)

### 🎯 What Just Happened?

The `@mlflow.trace` decorator:
1. **Captured** function inputs and outputs
2. **Measured** execution time
3. **Created** a span in the trace
4. **Logged** everything to MLflow

**No manual logging code needed!**

---
## Step 4: Span Types and Names

You can specify span types to categorize operations, and attach names to it by providing arguments to the decorator

In [4]:
from typing import List
# Specify span type and custom name
@mlflow.trace(name="document_retriever", span_type="RETRIEVER")
def retrieve_documents(query: str, top_k: int = 3) -> List[str]:
    """
    Simulate document retrieval from a vector database.
    """
    time.sleep(0.2)  # Simulate vector search
    
    # Simulated documents
    docs = [
        "MLflow is an open source platform for the ML and GenAI lifecycle and Observability .",
        "MLflow Tracing provides observability for GenAI applications.",
        "MLflow supports experiment tracking, model registry, and deployment."
    ]
    
    return docs[:top_k]

@mlflow.trace(name="query_embedder", span_type="EMBEDDING")
def embed_query(query: str) -> List[float]:
    """
    Generate embedding for query.
    """
    time.sleep(0.15)  # Simulate embedding generation
    
    # Simulated embedding
    return [0.1, 0.2, 0.3, 0.4, 0.5]

# Test both functions
print("\n📚 Testing typed spans...\n")

query = "What is MLflow?"

# These will create different span types
embedding = embed_query(query)
print(f"Embedding: {embedding[:3]}...")

documents = retrieve_documents(query, top_k=2)
print(f"\nRetrieved {len(documents)} documents")
print(f"  Doc 1: {documents[0][:50]}...")

print("\n✅ Typed spans created!")
print("\n🔍 In MLflow UI:")
print("   - embed_query: Type = EMBEDDING")
print("   - retrieve_documents: Type = RETRIEVER")
print("   - Easy to filter by operation type!")


📚 Testing typed spans...

Embedding: [0.1, 0.2, 0.3]...

Retrieved 2 documents
  Doc 1: MLflow is an open source platform for the ML and G...

✅ Typed spans created!

🔍 In MLflow UI:
   - embed_query: Type = EMBEDDING
   - retrieve_documents: Type = RETRIEVER
   - Easy to filter by operation type!


[Trace(trace_id=tr-61aa2e80625ae86b4fd7dfa507facae2), Trace(trace_id=tr-cb529ba7b6710a0f8429475372245e6d)]

### 📋 Standard Span Types

MLflow defines these standard types:

- `CHAIN`: Sequence of operations
- `LLM`: Language model call
- `RETRIEVER`: Document retrieval
- `EMBEDDING`: Text embedding
- `TOOL`: Tool/function execution
- `AGENT`: Agent reasoning
- `PARSER`: Output parsing

Use the appropriate type for better organization!

---
## Step 5: Adding Custom Attributes

Enrich spans with additional metadata.

In [5]:
# Add custom attributes to spans
@mlflow.trace(name="enhanced_retriever", span_type="RETRIEVER")
def enhanced_retrieve(query: str, top_k: int = 3) -> List[str]:
    """
    Retriever with rich metadata logging.
    """
    span = mlflow.get_current_active_span()
    # set attributes
    span.set_attributes({"top_k":top_k,
                        "query_length":len(query),
                        "search_method":"vector_similarity"})

    # Simulate retrieval
    time.sleep(0.2)
    docs = [
        "MLflow is an open source platform.",
        "MLflow provides tracing capabilities.",
        "MLflow supports multiple frameworks."
    ]
    
    # set attributes
    span.set_attributes({"num_results": len(docs[:top_k]),
                        "total_docs_in_db": 1000})  # Simulated
    
    return docs[:top_k]

# Test it
print("\n📊 Testing custom attributes...\n")

docs = enhanced_retrieve("What is MLflow tracing?", top_k=2)

print(f"Retrieved {len(docs)} documents")
print("\n✅ Span created with custom attributes!")
print("\n🔍 Custom attributes visible in UI:")
print("   - top_k: 2")
print("   - query_length: 24")
print("   - search_method: vector_similarity")
print("   - num_results: 2")
print("   - total_docs_in_db: 1000")


📊 Testing custom attributes...

Retrieved 2 documents

✅ Span created with custom attributes!

🔍 Custom attributes visible in UI:
   - top_k: 2
   - query_length: 24
   - search_method: vector_similarity
   - num_results: 2
   - total_docs_in_db: 1000


Trace(trace_id=tr-1711e4424769ca83f524e69f3ddb70b0)

### 💡 When to Add Custom Attributes

Add attributes for:
- **Configuration** (top_k, threshold, model_name)
- **Performance metrics** (num_results, cache_hit)
- **Data characteristics** (query_length, doc_size)
- **Business logic** (user_tier, feature_flags)
- **Debugging info** (data_source, version)

These make traces **searchable and analyzable**!

---
## Step 6: Building Hierarchical Traces

Create parent-child relationships between spans, using a CHAIN span type

In [6]:
from typing import Dict, List
# Build a hierarchical trace
@mlflow.trace(name="parse_query", span_type="PARSER")
def parse_query(query: str) -> Dict:
    """Extract intent and entities from query."""
    time.sleep(0.05)
    return {
        "intent": "question",
        "entities": ["MLflow"],
        "cleaned_query": query.strip()
    }

@mlflow.trace(name="embed_text", span_type="EMBEDDING")
def embed_text(text: str) -> List[float]:
    """Generate embedding."""
    time.sleep(0.1)
    return [0.1] * 768  # Simulated embedding

@mlflow.trace(name="vector_search", span_type="RETRIEVER")
def vector_search(embedding: List[float], top_k: int = 3) -> List[Dict]:
    """Search vector database."""
    span = mlflow.get_current_active_span()
    span.set_attributes({"embedding_dim": len(embedding),
                         "top_k":top_k})
    
    time.sleep(0.15)
    return [
        {"text": "MLflow is an open source platform.", "score": 0.95},
        {"text": "MLflow provides tracing.", "score": 0.89},
        {"text": "MLflow supports deployments.", "score": 0.82}
    ][:top_k]

# Test the hierarchical pipeline with a CHAIN span type
@mlflow.trace(name="rag_retrieval_pipeline", span_type="CHAIN")
def rag_retrieval_pipeline(user_query: str, top_k: int = 3) -> List[Dict]:
    """
    Complete retrieval pipeline with multiple steps.
    This parent span will contain all child spans!
    """
    span = mlflow.get_current_active_span()
    span.set_attributes({"pipeline_version":"v1.0",
                         "top_k":top_k})
    
    # Step 1: Parse query (child span)
    parsed = parse_query(user_query)
    
    # Step 2: Embed query (child span)
    embedding = embed_text(parsed["cleaned_query"])
    
    # Step 3: Vector search (child span)
    results = vector_search(embedding, top_k=top_k)
    span.set_attributes({"num_results": len(results)})
    
    return results

# Test the hierarchical pipeline
print("\n🌳 Testing hierarchical tracing...\n")

results = rag_retrieval_pipeline("What is MLflow?", top_k=2)

print(f"Retrieved {len(results)} documents:")
for i, result in enumerate(results, 1):
    print(f"  {i}. {result['text'][:40]}... (score: {result['score']})")

print("\n✅ Hierarchical trace created!")
print("\n🌳 Trace hierarchy:")
print("""
   rag_retrieval_pipeline (CHAIN)
   ├── parse_query (PARSER)
   ├── embed_text (EMBEDDING)
   └── vector_search (RETRIEVER)
""")
print("View the timeline in MLflow UI to see the hierarchy!")


🌳 Testing hierarchical tracing...

Retrieved 2 documents:
  1. MLflow is an open source platform.... (score: 0.95)
  2. MLflow provides tracing.... (score: 0.89)

✅ Hierarchical trace created!

🌳 Trace hierarchy:

   rag_retrieval_pipeline (CHAIN)
   ├── parse_query (PARSER)
   ├── embed_text (EMBEDDING)
   └── vector_search (RETRIEVER)

View the timeline in MLflow UI to see the hierarchy!


Trace(trace_id=tr-aac05ce4ea849093d3fb49718889f9e9)

### 🌳 Hierarchical Tracing Benefits

**Advantages:**
- **Logical organization** of complex workflows
- **Performance analysis** per operation
- **Easy debugging** - identify which step failed
- **Clear dependencies** between operations
- **Aggregate metrics** at parent level

**The parent span automatically:**
- Contains all child spans
- Calculates total duration
- Preserves execution order
- Shows the call stack

---
## Step 7: Complete RAG Pipeline with Tracing

Let's build a full RAG pipeline combining manual tracing and autologging.

In [7]:
# Complete RAG implementation with comprehensive tracing

@mlflow.trace(name="context_retrieval", span_type="RETRIEVER")
def retrieve_context(query: str, top_k: int = 3) -> List[str]:
    """
    Retrieve relevant context documents.
    """
    span = mlflow.get_current_active_span()
    span.set_attributes({"retrieval_method":"semantic_search",
                         "top_k":top_k})

    
    # Simulate retrieval
    time.sleep(0.2)
    
    # fake docs retrieved
    docs = [
        "MLflow is an open source platform for managing ML and GenAI lifecycle.",
        "MLflow Tracing captures LLM execution with detailed spans.",
        "MLflow integrates with OpenAI, LangChain, and 30+ frameworks.",
        "MLflow provides experiment tracking and model registry.",
        "MLflow supports deployment to various serving platforms."
    ]
    
    retrieved = docs[:top_k]
    span.set_attributes({"num_docs_retrieved": len(retrieved)})
    
    return retrieved

@mlflow.trace(name="format_prompt", span_type="PARSER")
def format_rag_prompt(query: str, context_docs: List[str]) -> str:
    """
    Format RAG prompt with context and query.
    """
    span = mlflow.get_current_active_span()
    span.set_attributes({"num_context_docs": len(context_docs)})
    
    context = "\n".join([f"- {doc}" for doc in context_docs])
    
    prompt = f"""Use the following context to answer the question:

Context:
{context}

Question: {query}

Answer concisely based on the context above:"""
    span.set_attributes({"prompt_length": len(prompt)})
    span.set_attributes({"prompt": prompt})
    span.set_attributes({"query": query})
    span.set_attributes({"context": context})
    
    return prompt

@mlflow.trace(name="generate_answer", span_type="LLM")
def generate_answer(prompt: str, model: str = "gpt-5-mini") -> str:
    """
    Generate answer using LLM.
    OpenAI call will be auto-traced!
    """
    span = mlflow.get_current_active_span()
    span.set_attributes({"model": model})
    
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=1.0,
        max_tokens=2000
    )
    
    answer = response.choices[0].message.content
    
    span.set_attributes({"answer_length": len(answer),
                         "tokens_used": response.usage.total_tokens})
    
    return answer

# Complete RAG implementation with comprehensive tracing
# The entire pipeline is traced manually
@mlflow.trace(name="rag_qa_system", span_type="CHAIN")
def rag_qa_system(user_query: str, top_k: int = 3) -> Dict:
    """
    Complete RAG question-answering system.
    
    Returns:
        Dict with answer, context, and metadata
    """
    span = mlflow.get_current_active_span()
    span.set_attributes({"system_version": "v1.0",
                         "user_query": user_query})
    
    # Step 1: Retrieve context (manually traced)
    context_docs = retrieve_context(user_query, top_k=top_k)
    
    # Step 2: Format prompt (manually traced)
    prompt = format_rag_prompt(user_query, context_docs)
    
    # Step 3: Generate answer (manually + auto traced)
    answer = generate_answer(prompt, model_name)
    
    result = {
        "query": user_query,
        "answer": answer,
        "context_docs": context_docs,
        "num_docs_used": len(context_docs)
    }
    span.set_attributes({"answer_generated": True})
    
    
    return result

# Test the complete RAG system
print("\n🤖 Testing complete RAG system with full tracing...\n")

result = rag_qa_system("What tracing capabilities does MLflow provide?", top_k=3)

print(f"Query: {result['query']}")
print(f"\nContext docs used: {result['num_docs_used']}")
print(f"\nAnswer:\n{result['answer']}")

print("\n" + "="*60)
print("✅ RAG System Complete with Full Observability!")
print("="*60)

print("""
\n🔍 Trace Hierarchy in MLflow UI:

rag_qa_system (CHAIN)
├── context_retrieval (RETRIEVER)
├── format_prompt (PARSER)
└── generate_answer (LLM)
    └── OpenAI API call (auto-traced)

Each span includes:
✓ Inputs and outputs
✓ Duration and timing
✓ Custom attributes
✓ Error information (if any)
""")


🤖 Testing complete RAG system with full tracing...

Query: What tracing capabilities does MLflow provide?

Context docs used: 3

Answer:
MLflow provides **MLflow Tracing**, which captures **LLM execution traces with detailed spans**, and it works across integrations like **OpenAI, LangChain, and 30+ other frameworks**.

✅ RAG System Complete with Full Observability!


🔍 Trace Hierarchy in MLflow UI:

rag_qa_system (CHAIN)
├── context_retrieval (RETRIEVER)
├── format_prompt (PARSER)
└── generate_answer (LLM)
    └── OpenAI API call (auto-traced)

Each span includes:
✓ Inputs and outputs
✓ Duration and timing
✓ Custom attributes
✓ Error information (if any)



Trace(trace_id=tr-f88b499d7d60fb95f9370748d96a5aa9)

### 🎯 RAG Pipeline Analysis

With this tracing setup, you can:

1. **Identify bottlenecks**: Which step takes longest?
2. **Debug failures**: Which step failed and why?
3. **Optimize retrieval**: Is top_k=3 optimal?
4. **Track costs**: How many tokens per query?
5. **Monitor quality**: Are retrieved docs relevant?

**All visible in the MLflow UI timeline view!**

---
## Step 8: Tracing Agentic Workflows

Let's trace a simple agent with tool usage.

In [8]:
# Agentic workflow with tool usage
import json

@mlflow.trace(name="weather_tool", span_type="TOOL")
def get_weather(city: str) -> Dict:
    """
    Simulated weather tool.
    """
    span = mlflow.get_current_active_span()
    span.set_attributes({
        "tool_name": "weather_api",
        "city": city,
        "api_version": "v2.0"
    })
    
    time.sleep(0.1)  # Simulate API call
    
    weather_data = {
        "city": city,
        "temperature": "72°F",
        "condition": "Sunny",
        "humidity": "45%"
    }
    
    span.set_attributes({   
        "data_retrieved": True
    })
    
    return weather_data

# Another tool to use for the agent
@mlflow.trace(name="calculator_tool", span_type="TOOL")
def calculate(expression: str) -> float:
    """
    Simulated calculator tool.
    """
    span = mlflow.get_current_active_span()
    span.set_attributes({
        "tool_name": "calculator",
        "expression": expression
    })
    
    time.sleep(0.05)
    
    # UNSAFE: Don't use eval() in production!
    result = eval(expression)
    
    span.set_attributes({
        "result": result
    })
    
    return result

# The agent planning step in decideing which tool to use
# based on the user query
@mlflow.trace(name="agent_planning", span_type="AGENT")
def agent_plan(user_query: str) -> Dict:
    """
    Agent decides which tool to use.
    """
    span = mlflow.get_current_active_span()
    span.set_attributes({
        "user_query": user_query
    })
    
    # Use LLM to plan (auto-traced)
    system_prompt = """You are an agent with access to:
1. get_weather(city) - Get weather information
2. calculate(expression) - Evaluate math expressions

Decide which tool to use and extract parameters.
Respond with JSON: {"tool": "tool_name", "params": {...}}"""
    
    response = client.chat.completions.create(
        model=model_name,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_query}
        ],
        temperature=1.0
    )

    # Parse the response
    returned_plan = json.loads(response.choices[0].message.content)
    span.set_attributes({
        "returned_plan": returned_plan
    }) 
 
    print(f"returned_plan: {returned_plan}")
    
    # Simulate parsing (simplified)
    if "weather" in user_query.lower():
        plan = {"tool": "weather", "params": {"city": "San Francisco"}}
    else:
        plan = {"tool": "calculator", "params": {"expression": "42 * 2"}}
    
    span.set_attributes({
        "selected_tool": plan["tool"]
    })
    
    return plan

# run the agent in a chain
@mlflow.trace(name="agent_execution", span_type="CHAIN")
def run_agent(user_query: str) -> str:
    """
    Complete agent execution with planning and tool use.
    """
    span = mlflow.get_current_active_span()
    span.set_attributes({
        "agent_version": "v1.0",
        "user_query": user_query
    })
    
    # Step 1: Plan (uses LLM)
    plan = agent_plan(user_query)
    
    # Step 2: Execute tool
    if plan["tool"] == "weather":
        tool_result = get_weather(plan["params"]["city"])
    else:
        tool_result = calculate(plan["params"]["expression"])
    
    span.set_attributes({
        "tool_executed": plan["tool"]
    })
    
    # Step 3: Generate final response (uses LLM)
    response = client.chat.completions.create(
        model=model_name,
        messages=[{
            "role": "user",
            "content": f"Tool result: {tool_result}. Answer: {user_query}"
        }],
        temperature=1.0
    )
    
    final_answer = response.choices[0].message.content
    
    span.set_attributes({
        "answer_generated": True
    })
    
    return final_answer

# Test the agent
print("\n🤖 Testing agentic workflow with tool tracing...\n")

query1 = "What's the weather in San Francisco?"
answer1 = run_agent(query1)

print(f"Query: {query1}")
print(f"Answer: {answer1}\n")

query2 = "Calculate 42 times 2"
answer2 = run_agent(query2)

print(f"Query: {query2}")
print(f"Answer: {answer2}")

print("\n" + "="*60)
print("✅ Agent Workflow Fully Traced!")
print("="*60)

print("""
\n🔍 Agent Trace Hierarchy:

agent_execution (CHAIN) (manula trace)
├── agent_planning (AGENT) (manula trace)
│   └── OpenAI API call (auto-traced)
├── weather_tool or calculator_tool (TOOL) (manula trace)
└── OpenAI API call for final response (auto-traced)

Key insights visible:
✓ Which tool was selected
✓ Tool execution time
✓ Tool parameters and results
✓ Total agent reasoning time
✓ Token usage across all LLM calls
""")


🤖 Testing agentic workflow with tool tracing...

returned_plan: {'tool': 'get_weather', 'params': {'city': 'San Francisco'}}
Query: What's the weather in San Francisco?
Answer: In San Francisco, it’s **72°F** and **sunny**, with **45% humidity**.

returned_plan: {'tool': 'calculate', 'params': {'expression': '42*2'}}
Query: Calculate 42 times 2
Answer: 42 × 2 = **84**

✅ Agent Workflow Fully Traced!


🔍 Agent Trace Hierarchy:

agent_execution (CHAIN) (manula trace)
├── agent_planning (AGENT) (manula trace)
│   └── OpenAI API call (auto-traced)
├── weather_tool or calculator_tool (TOOL) (manula trace)
└── OpenAI API call for final response (auto-traced)

Key insights visible:
✓ Which tool was selected
✓ Tool execution time
✓ Tool parameters and results
✓ Total agent reasoning time
✓ Token usage across all LLM calls



[Trace(trace_id=tr-0d0aa107d690abd232a66b49e267d46e), Trace(trace_id=tr-cea93509d6594b27f63c075c07811313)]

### 🤖 Agent Tracing Benefits

For agentic workflows, tracing reveals:

1. **Decision Making**: Which tool was chosen and why?
2. **Tool Performance**: How long does each tool take?
3. **Error Tracking**: Which tool failed?
4. **Cost Analysis**: How many LLM calls per agent run?
5. **Optimization**: Can we skip unnecessary steps?

**Essential for debugging complex agents!**

---
## Step 9: Advanced Debugging Techniques

Let's explore how to use traces for debugging, besides inspection and retrospection.
Note that we are using manual tracing, expilicty by using the decorator

In [9]:
# Debugging example: Trace a buggy function, which can raise an error

@mlflow.trace(name="buggy_processor", span_type="PARSER")
def process_with_validation(data: Dict) -> Dict:
    """
    Fake function with potential errors.
    """
    span = mlflow.get_current_active_span()
    span.set_attributes({
        "input_keys": list(data.keys())
    })
    
    # Log intermediate state for debugging
    span.set_attributes({
        "validation_step": "checking_required_fields"
    })
    
    # This might raise an error
    if "required_field" not in data:
        span.set_attributes({
            "error_type": "missing_field"
        })
        raise ValueError("Missing required_field")
    span.set_attributes({
        "validation_step": "passed"
    })
    
    # Process data
    result = {"processed": True, "value": data.get("required_field")}
    
    return result

print("\n🐛 Testing error tracing...\n")

# Success case
try:
    result = process_with_validation({"required_field": "test"})
    print(f"✅ Success case: {result}")
except Exception as e:
    print(f"❌ Error: {e}")

# Failure case
print("\nTesting failure case...")
try:
    result = process_with_validation({"wrong_field": "test"})
    print(f"✅ Success case: {result}")
except Exception as e:
    print(f"❌ Error: {e}")
    print("\n🔍 Error details captured in trace!")

print("""
\n💡 Debugging with Traces:

1. Filter traces by error status in UI
2. View the failed span (marked in red)
3. Check custom attributes:
   - What were the inputs?
   - What validation step failed?
   - What was the error type?
4. Compare with successful traces
5. Reproduce the issue with exact inputs
6. Use MLflow Assistant to get insights on the failed traces and root cause

Attributes logged BEFORE the error help identify root cause!
""")


🐛 Testing error tracing...

✅ Success case: {'processed': True, 'value': 'test'}

Testing failure case...
❌ Error: Missing required_field

🔍 Error details captured in trace!


💡 Debugging with Traces:

1. Filter traces by error status in UI
2. View the failed span (marked in red)
3. Check custom attributes:
   - What were the inputs?
   - What validation step failed?
   - What was the error type?
4. Compare with successful traces
5. Reproduce the issue with exact inputs
6. Use MLflow Assistant to get insights on the failed traces and root cause

Attributes logged BEFORE the error help identify root cause!



[Trace(trace_id=tr-f530682c405447351f5a1e365ab7c555), Trace(trace_id=tr-8b89f966c02cbbe9821e4a3d123c25fa)]

---
## Step 10: Performance Analysis

Use traces to identify performance bottlenecks.

In [10]:
# Performance analysis example
print("""
╔══════════════════════════════════════════════════════════════╗
║         Performance Analysis with Traces                     ║
╚══════════════════════════════════════════════════════════════╝

In the MLflow UI, you can:

1. 📊 TIMELINE VIEW:
   - See which operations take the most time
   - Identify serial vs parallel operations
   - Find bottlenecks visually

2. 📈 AGGREGATE METRICS:
   - Average latency per span type
   - P50, P95, P99 latencies
   - Success rate per operation

3. 🔍 COMPARISON:
   - Compare traces before/after optimization
   - A/B test different implementations
   - Track performance over time

4. 💡 OPTIMIZATION STRATEGIES:
   
   If retrieval is slow:
   - Check embedding generation time
   - Optimize vector search
   - Consider caching
   
   If LLM calls are slow:
   - Reduce max_tokens
   - Use streaming responses
   - Try smaller models
   
   If overall latency is high:
   - Parallelize independent operations
   - Cache frequent queries
   - Optimize prompt length

5. 📊 METRICS TO TRACK:
   - End-to-end latency
   - Per-operation latency
   - Token usage
   - Cache hit rate
   - Error rate

""")

# Run a simple performance test
print("Running performance test with 3 queries...\n")

test_queries = [
    "What is MLflow's GenAI offering?",
    "How does tracing work?",
    "What are the benefits?"
]

for i, query in enumerate(test_queries, 1):
    print(f"Query {i}/3: {query[:30]}...")
    start = time.time()
    result = rag_qa_system(query, top_k=2)
    latency = time.time() - start
    print(f"  Latency: {latency:.2f}s\n")

print("✅ Performance test complete!")
print("\n🔍 Compare these traces in the UI:")
print("   - Are latencies consistent?")
print("   - Which steps vary the most?")
print("   - Any outliers to investigate?")


╔══════════════════════════════════════════════════════════════╗
║         Performance Analysis with Traces                     ║
╚══════════════════════════════════════════════════════════════╝

In the MLflow UI, you can:

1. 📊 TIMELINE VIEW:
   - See which operations take the most time
   - Identify serial vs parallel operations
   - Find bottlenecks visually

2. 📈 AGGREGATE METRICS:
   - Average latency per span type
   - P50, P95, P99 latencies
   - Success rate per operation

3. 🔍 COMPARISON:
   - Compare traces before/after optimization
   - A/B test different implementations
   - Track performance over time

4. 💡 OPTIMIZATION STRATEGIES:

   If retrieval is slow:
   - Check embedding generation time
   - Optimize vector search
   - Consider caching

   If LLM calls are slow:
   - Reduce max_tokens
   - Use streaming responses
   - Try smaller models

   If overall latency is high:
   - Parallelize independent operations
   - Cache frequent queries
   - Optimize prompt length

5

[Trace(trace_id=tr-761396cca83a8308fb8b315a1d4438a3), Trace(trace_id=tr-a7127daf40f75640e9e83d289822e307), Trace(trace_id=tr-2ef454b8e778200420b508ba9b8c564a)]

---
## Summary

In this notebook, you learned:

1. ✅ When to use manual tracing vs. autologging
2. ✅ The `@mlflow.trace` decorator for custom functions
3. ✅ Span types and naming conventions
4. ✅ Adding custom attributes with `span = mlflow.get_current_active_span(); span.set_attributes({...})`
5. ✅ Building hierarchical traces (parent-child relationships)
6. ✅ Tracing complete RAG pipelines end-to-end
7. ✅ Tracing agentic workflows with tool usage
8. ✅ Advanced debugging techniques with traces
9. ✅ Performance analysis and optimization strategies

### Key Takeaways

- **Combine** autologging and manual tracing for complete coverage
- **Use span types** for better organization and filtering
- **Add custom attributes** for debugging and analysis
- **Build hierarchies** to represent complex workflows accurately
- **Trace everything** from retrieval to generation to validation
- **Use traces** for debugging, performance, and optimization

### Production Best Practices

1. **Always trace** in development and staging
2. **Sample traces** in high-volume production (e.g., 1-10%)
3. **Monitor key metrics** from traces (latency, errors, costs)
4. **Set up alerts** for anomalies
5. **Review traces** during incident response
6. **Use trace IDs** for cross-system correlation

### What's Next?

**📓 Notebook 1.5: Prompt Management**

Learn how to:
- Create reusable prompt templates
- Version prompts systematically
- Link prompts to experiments and traces
- Share prompts across your team
- Track prompt performance